<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Conclusions</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#2-Nodes" data-toc-modified-id="2-Nodes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>2 Nodes</a></span><ul class="toc-item"><li><span><a href="#Is-the-init-correct?" data-toc-modified-id="Is-the-init-correct?-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Is the init correct?</a></span></li></ul></li></ul></div>

# Goal

We wanted to resolve the initialization, that happened to be a bit weird. Indeed the rebalancing edges were not properly initialized and the costs were not going steadily down. 

# Conclusions

The approach was good, but we forgot to enforce that the "available" rebalancing edges be balanced at the beginning. 

# Load

In [39]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd

from amod_ed.result_analysis import plot_ri, print_final_flows, plot_cost_all_path, plot_ri_list, plot_balance_list
from amod_ed.result_analysis import plot_stop_and_cost

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
subfolder='unstucking_balance'

# 2 Nodes

In [49]:
n_nodes=2

In [50]:
path='Data/'+str(n_nodes)+'Nodes_back2basics/outputs/'+subfolder

In [51]:
print(os.listdir(path))

['new_init.pkl', 'initial_setup.pkl']


In [52]:
filename=os.listdir(path)[1]
print(filename)

with open(os.path.join(path, filename), 'rb') as f:
        G_FW1, OD, ri_FW1, n_outer, n_inner, balance, opt_res, OD_list, balance_list, params = pickle.load(f)

initial_setup.pkl


In [53]:
filename=os.listdir(path)[0]
print(filename)

with open(os.path.join(path, filename), 'rb') as f:
        G_FW2, OD, ri_FW2, n_outer, n_inner, balance, opt_res, OD_list, balance_list, params = pickle.load(f)

new_init.pkl


## Is the init correct?

Now it is!!

Currently we are solving the following problem: 
$$
\min_{f} \|f-f_r\| \\
\text{s.t.    } A_{in} f - A_{out} f = r\\
\text{AND   } f_R = (\sum_{in} - \sum_{out}) f_o
$$

I had forgotten to include 
$$ f \geq 0 $$
as a constraint. 

The last equation simply means that the flow on the rebalancing edge must be equal to the net rebalancing flow at the origin node of that edge. 

**Reminder**: how are the ri computed? 
- Initialization of all ri to 0
- We loop through the nodes
    - for the origin of the edge, we add the flow
    - for the destination of the edge, we substract the flow

Therefore, 
- A node with ri>0 is a node that is in deficit of rebalancers
- A node with ri<0 is a node that is in excess of rebalancers


**Reminder**: how do we update OD? 
- We add a pair $(n,R)$ for each node $n$ that is in excess. 